# **Face Recognition Based Attendance**

In [ ]:
# Install required libraries
!pip install Pillow
!pip install google-colab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.0 MB/s 


In [ ]:
# Import required libraries
from keras.models import load_model
from keras.preprocessing import image
from IPython.display import Image, display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image, ImageOps
from datetime import datetime
import numpy as np
import os

In [ ]:
# Google Sheets integration
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
worksheet = gc.open('Attendance').sheet1

In [ ]:
# Camera Setup
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      await new Promise((resolve) => capture.onclick = resolve);
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

# Image Capture
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  display(Image(filename))

# Error display
except Exception as err:
  print(str(err))

<IPython.core.display.Javascript object>

Saved to photo.jpg
'module' object is not callable


In [ ]:
# Load the model
model = load_model('keras_model.h5')
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
image = Image.open('photo.jpg')
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)
image_array = np.asarray(image)
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
data[0] = normalized_image_array

# Load current time
now = datetime.now()
ctime = now.strftime("%d/%m/%Y %H:%M:%S")
i=2

In [ ]:
# Generate prediction
prediction = model.predict(data)
print(prediction)
if prediction[0,0] > 0.6:
    student = "Person 1"
    worksheet.update_cell(2,i,ctime)
if prediction[0,1] > 0.6:
    student = "Person 2"
    worksheet.update_cell(3,i,ctime)
if prediction[0,2] > 0.6:
    student = "Person 3"
    worksheet.update_cell(4,i,ctime)

print(student," Present")
i=i+1

1/1 [==============================] - 1s 850ms/step
[[0.08453184 0.13307792 0.78239024]]
Yathin  Present
